In [1]:
# https://stackoverflow.com/questions/29760935/how-to-get-vector-for-a-sentence-from-the-word2vec-of-tokens-in-sentence

In [2]:
import pandas as pd
import gensim

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
def normalize_and_add_pos(word):
    p = morph.parse(word)[0]
    return '{}_{}'.format(
        p.normal_form,
        p.tag.POS
    )

In [5]:
data = pd.read_csv('80k_new_3.csv', sep = ';')

In [6]:
data.drop(data.columns[[2]], axis=1, inplace=True)

In [77]:
data

,moderated,text,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,0,Говнояндекс,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,В чем проблема ?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,ямы делают?,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Встаёт отсюда,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Привет,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,Две недели!,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0,Объясните почему??! Бля,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1,Дам 2 проездных ),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1,че не едет ни...а???,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0,С наступающими)) скоро пиздец из пробок будет ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 1 - не спам, 0 - спам

In [7]:
y = data['moderated']

In [8]:
X = data['text']

In [9]:
X = X.fillna('слово')
new_X = []

In [10]:
for i in X:
    arr = []
    sent = i.split()
    for word in sent:
        arr.append(word.strip('.!?'))
    new_X.append(arr)

In [11]:
from gensim.models.keyedvectors import KeyedVectors

In [12]:
w_model = gensim.models.Word2Vec()

In [13]:
w_model.build_vocab(new_X)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(new_X, y, test_size=0.33, random_state=42)

In [15]:
y_train = list(y_train)

In [16]:
y_test = list(y_test)

In [17]:
norm_X_train = []
norm_y_train = []
i = -1
for text in X_train:
    i+=1
    text_vectors = []
    
    for word in text:
#         norm_word = normalize_and_add_pos(word)
#         if norm_word in w_model:
        if word in w_model:
            text_vectors.append(w_model[word])
        else:
            pass
    try:
        mean_vector = sum(text_vectors) / len(text_vectors)
        norm_X_train.append(mean_vector)
        norm_y_train.append(y_train[i])
    except:
        pass
#         print('no')

In [18]:
len(norm_X_train)

27904

In [19]:
len(norm_y_train)

27904

In [20]:
norm_X_test = []
norm_y_test = []
i = -1
for text in X_test:
    i+=1
    text_vectors = []
    
    
    for word in text:
#         norm_word = normalize_and_add_pos(word)
#         if norm_word in w_model:
        if word in w_model:
            text_vectors.append(w_model[word])
            
    try:
        mean_vector = sum(text_vectors) / len(text_vectors)
        norm_X_test.append(mean_vector)
        norm_y_test.append(int(y_test[i]))
    except:
        pass

In [21]:
np_X_train = np.array(norm_X_train)
np_X_train.shape

(27904, 100)

In [22]:
np_y_train = np.array(norm_y_train)
np_y_train.shape

(27904,)

In [23]:
from keras.utils.np_utils import to_categorical
y_binary = to_categorical(np_y_train)
y_binary.shape

Using TensorFlow backend.


(27904, 2)

In [24]:
np_X_test = np.array(norm_X_test)
np_X_test.shape

(13801, 100)

In [25]:
np_y_test = np.array(norm_y_test)
np_y_test.shape

(13801,)

In [26]:
y_test_binary = to_categorical(np_y_test)
y_test_binary.shape

(13801, 2)

In [27]:
import keras.backend as K
def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    # How many relevant items are selected?
    recall = c1 / c3

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

In [101]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam
from keras.layers.convolutional import Conv1D

model = Sequential()
model.add(Dense(500, input_shape=(100,), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(100, activation='tanh'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))
model.add(Activation('elu'))

# adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

model.compile(loss='categorical_crossentropy',
              optimizer='nadam',
#               optimizer=adam,
              metrics=[f1_score])

history = model.fit(np_X_train, y_binary,
                    batch_size=32,
                    epochs=15,
                    verbose=1,
                    validation_split=0.2)

Train on 22323 samples, validate on 5581 samples
Epoch 1/15
22323/22323 [==============================] - 4s - loss: 0.5955 - f1_score: 0.7096 - val_loss: 0.5784 - val_f1_score: 0.7160

In [102]:
score, acc = model.evaluate(np_X_test, y_test_binary,
                            batch_size=32)

print('Test accuracy:', acc)

12896/13801 [===========================>..] - ETA: 0sTest accuracy: 0.731758568224


In [103]:
y_pred = model.predict(np_X_test)

In [104]:
fin_y_pred = []
for i in y_pred:
    fin_y_pred.append(arr.index(max(arr)))
    

In [105]:
from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(np_y_test, fin_y_pred))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00      4057
          1       0.71      1.00      0.83      9744

avg / total       0.50      0.71      0.58     13801



/Users/Valeriya/anaconda/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [79]:
def predict_from_sentence(sentence):
    words = []
    sent = sentence.split()
    for word in sent:
        words.append(word.strip('.!?'))
    
    vectors = []
    for word in words:
        try:
            vec = w_model.wv[word]
            vectors.append(vec)
        except:
            continue
    try:
        text_vec = sum(vectors) / len(vectors)
        vec = np.array([text_vec])
        res = model.predict(vec)[0]  
        arr = res.tolist()
        
        return arr.index(max(arr))
    except:
        return 'No text vector'   

In [88]:
predict_from_sentence('Идиоты блеать')

1

In [55]:
vec = np.array([vec])

In [64]:
res = model.predict(vec)[0]

In [66]:
arr = res.tolist()
arr.index(max(arr))

1